In [1]:
import requests
import pandas as pd

data = pd.read_csv("./sorted_playkuround.csv")
for i in range(data.shape[0]):
    searching = data['이름'].iloc[i]
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?x=37.588194705681&y=127.03402453668&query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK ea1128b9fb01a06cc80760f350042441"
    }
    try : 
        data['위도'].iloc[i] = requests.get(url, headers = headers).json()['documents'][0]['y']
        data['경도'].iloc[i] = requests.get(url, headers = headers).json()['documents'][0]['x']
        data['전화번호'].iloc[i] = requests.get(url, headers = headers).json()['documents'][0]['phone']
        data['주소'].iloc[i] = requests.get(url, headers = headers).json()['documents'][0]['address_name']
    except IndexError: 
        pass
data.to_csv("temp_playkuround.csv", mode='w')


/Users/gim-eungi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('seoulMarket.csv', sep=',', encoding='utf-8')

In [3]:
df = df.loc[df['상권업종대분류명'] == '음식']  

df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

df = df.loc[(df['행정동명'] == '안암동')]


In [4]:
df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

In [15]:
df.head

<bound method NDFrame.head of                name  cate_1       cate_2                   cate_3 dong  \
141            본도시락   패스트푸드       도시락전문점                  한식 음식점업  안암동   
1177           카페비원  커피점/카페  커피전문점/카페/다방                 비알콜 음료점업  안암동   
2167         고대프로방스      양식    기타전문서양음식점                 서양식 음식점업  안암동   
2441         뉴욕앤핫도그   패스트푸드          핫도그  피자, 햄버거, 샌드위치 및 유사 음식점업  안암동   
3368           달콤퐁당  커피점/카페  커피전문점/카페/다방                 비알콜 음료점업  안암동   
...             ...     ...          ...                      ...  ...   
307213  왕십리인생곱창연탄구이      한식     곱창/양구이전문                  한식 음식점업  안암동   
310995     도로시파스타연정      양식      스파게티전문점                 서양식 음식점업  안암동   
311536           고우  일식/수산물       음식점-일식                  일식 음식점업  안암동   
312192         스시판다  일식/수산물     음식점-초밥전문                  일식 음식점업  안암동   
312249         물가타로  커피점/카페         사주카페                 비알콜 음료점업  안암동   

              lon         lat  
141     37.582278  127.028589  
1177    37.587486

In [5]:
df['cate_mix'] = df['cate_1'] + df['cate_2'] +  df['cate_3']
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")



In [6]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]


이렇게 하면, 각 데이터 vs 데이터 로 서로 카테고리 텍스트가 얼마나 유사한지를 따질 수 있다.
이 데이터를 바탕으로 마지막 단계에 최종 채점 단계에서 활용할 것

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time


chromedriver = './chromedriver' 
driver = webdriver.Chrome(chromedriver) 


df['kakao_keyword'] = df['dong'] + "%20" + df['name']
df['kakao_map_url'] = ''


for i, keyword in enumerate(df['kakao_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword)
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # child(1)이 없던데요?
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass

이번에 찾을 키워드 : 0 / 401 행 안암동%20본도시락
이번에 찾을 키워드 : 1 / 401 행 안암동%20카페비원
이번에 찾을 키워드 : 2 / 401 행 안암동%20고대프로방스
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=98.0.4758.80)

이번에 찾을 키워드 : 3 / 401 행 안암동%20뉴욕앤핫도그
이번에 찾을 키워드 : 4 / 401 행 안암동%20달콤퐁당
이번에 찾을 키워드 : 5 / 401 행 안암동%20고려대학교교육관로즈버드커피전문점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=98.0.4758.80)

이번에 찾을 키워드 : 6 / 401 행 안암동%20디어브레드
이번에 찾을 키워드 : 7 / 401 행 안암동%20뚜또오케이고대점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=98.0.4758.80)

이번에 찾을 키워드 : 8 / 401 행 안암동%20카페낢진
이번에 찾을 키워드 : 9 / 401 행 안암동%

KeyboardInterrupt: 